In [1]:
import glam
import pandas as pd
import numpy as np
import os.path
import arviz as az

import matplotlib.pyplot as plt

/Users/pradyumna/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
import pymc3 as pm

In [3]:
np.random.seed(23) # from random.org

# Hierarchical GLAM estimation and out of sample prediction
## eLife reanalysis

## Load data

In [4]:
# Load data
sufix = '_hierarchical_Less_Bin_Gamma-11_NUTS_33_eLife'
data = pd.read_csv('data/PF2019_data/GlamDataPF2019_Less_Bin_33.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data.head()

subject  trial  choice        rt  item_value_0  item_value_1    gaze_0  \
0        1      0       0  4261.735             6             7  0.603448   
1        1      1       1  3559.258             0             0  0.490772   
2        1      2       1  3754.464             0             0  0.490893   
3        1      3       0  2431.751             2             0  0.639125   
4        1      4       0  2199.342             0             0  0.702232   

     gaze_1  
0  0.396552  
1  0.509228  
2  0.509107  
3  0.360875  
4  0.297768

## Split data in training and test sets

In [5]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

#test_data.to_csv(str('data/PF2019_data/GlamDataPF2019_preprocessed_test'+sufix+'.csv'))
#train_data.to_csv(str('data/PF2019_data/GlamDataPF2019_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

Split data into training (1920 trials) and test (1920 trials) sets...


In [6]:
train_data.subject.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33])

In [7]:
train_data.subject.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33])

In [8]:
# we renumber subject data for proper sequence
train_data2 = train_data.replace(train_data.subject.unique(), list(range(len(train_data.subject.unique()))))

In [9]:
train_data2.subject.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

## Hierarchical GLAM estimation

### 1. full GLAM

In [10]:
# Fitting full GLAM
print('Fitting full GLAM hierarchically...')

glam_full = glam.GLAM(train_data2)

if not os.path.exists(str('results/estimates/glam_PF2019_full_hierarchical_cv'+sufix+'.npy')):
    glam_full.make_model('hierarchical', gamma_bounds=(-1, 1), t0_val=0)
    glam_full.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_full.estimates = np.load(str('results/estimates/glam_PF2019_full_hierarchical_cv'+sufix+'.npy'))   

Fitting full GLAM hierarchically...
Generating hierarchical model for 32 subjects...


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)

Fitting 1 model(s) using NUTS...
  Fitting model 1 of 1...


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, tau_sd, tau_mu, SNR, SNR_sd, SNR_mu, gamma, gamma_sd, gamma_mu, v, v_sd, v_mu]


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)

/!\ Automatically setting parameter precision...


In [11]:
# Save parameter estimates
np.save(str('results/estimates/glam_PF2019_nobias_hierarchical_cv'+sufix+'.npy'), glam_full.estimates)
pd.DataFrame(glam_full.estimates)

b  p_error      v_mu      v_sd         v  gamma_mu  gamma_sd  gamma  \
0   1.0     0.05  0.000044  0.000023  0.000047     -0.64       0.1  -0.83   
1   1.0     0.05  0.000044  0.000023  0.000019     -0.64       0.1  -0.95   
2   1.0     0.05  0.000044  0.000023  0.000045     -0.64       0.1  -0.64   
3   1.0     0.05  0.000044  0.000023  0.000027     -0.64       0.1  -0.58   
4   1.0     0.05  0.000044  0.000023  0.000027     -0.64       0.1  -0.56   
5   1.0     0.05  0.000044  0.000023  0.000066     -0.64       0.1  -0.62   
6   1.0     0.05  0.000044  0.000023  0.000035     -0.64       0.1  -0.57   
7   1.0     0.05  0.000044  0.000023  0.000047     -0.64       0.1  -0.59   
8   1.0     0.05  0.000044  0.000023  0.000112     -0.64       0.1  -0.53   
9   1.0     0.05  0.000044  0.000023  0.000059     -0.64       0.1  -0.52   
10  1.0     0.05  0.000044  0.000023  0.000030     -0.64       0.1  -0.81   
11  1.0     0.05  0.000044  0.000023  0.000082     -0.64       0.1  -0.70   
12  1.0     0.05  0.000044  0.000023  0.000034     -0.64       0.1  -0.71   
13  1.0     0.05  0.000044  0.000023  0.000034     -0.64       0.1  -0.58   
14  1.0     0.05  0.000044  0.000023  0.000046     -0.64       0.1  -0.57   
15  1.0     0.05  0.000044  0.000023  0.000049     -0.64       0.1  -0.53   
16  1.0     0.05  0.000044  0.000023  0.000023     -0.64       0.1  -0.63   
17  1.0     0.05  0.000044  0.000023  0.000030     -0.64       0.1  -0.63   
18  1.0     0.05  0.000044  0.000023  0.000043     -0.64       0.1  -0.83   
19  1.0     0.05  0.000044  0.000023  0.000047     -0.64       0.1  -0.58   
20  1.0     0.05  0.000044  0.000023  0.000034     -0.64       0.1  -0.60   
21  1.0     0.05  0.000044  0.000023  0.000041     -0.64       0.1  -0.54   
22  1.0     0.05  0.000044  0.000023  0.000033     -0.64       0.1  -0.65   
23  1.0     0.05  0.000044  0.000023  0.000041     -0.64       0.1  -0.63   
24  1.0     0.05  0.000044  0.000023  0.000019     -0.64       0.1  -0.69   
25  1.0     0.05  0.000044  0.000023  0.000040     -0.64       0.1  -0.62   
26  1.0     0.05  0.000044  0.000023  0.000031     -0.64       0.1  -0.53   
27  1.0     0.05  0.000044  0.000023  0.000065     -0.64       0.1  -0.16   
28  1.0     0.05  0.000044  0.000023  0.000085     -0.64       0.1  -0.55   
29  1.0     0.05  0.000044  0.000023  0.000068     -0.64       0.1  -0.63   
30  1.0     0.05  0.000044  0.000023  0.000043     -0.64       0.1  -0.60   
31  1.0     0.05  0.000044  0.000023  0.000045     -0.64       0.1  -0.67   

    SNR_mu  SNR_sd     SNR         s  tau_mu  tau_sd   tau   t0  
0   191.63    67.8  172.99  0.007310    0.76     0.5  0.65  0.0  
1   191.63    67.8  281.24  0.005324    0.76     0.5  1.06  0.0  
2   191.63    67.8  202.53  0.008786    0.76     0.5  0.10  0.0  
3   191.63    67.8  364.38  0.009589    0.76     0.5  1.32  0.0  
4   191.63    67.8  227.65  0.006119    0.76     0.5  0.84  0.0  
5   191.63    67.8  111.33  0.007750    0.76     0.5  0.53  0.0  
6   191.63    67.8  271.66  0.008658    0.76     0.5  1.44  0.0  
7   191.63    67.8  119.14  0.010842    0.76     0.5  0.21  0.0  
8   191.63    67.8   45.70  0.005171    0.76     0.5  0.00  0.0  
9   191.63    67.8  176.17  0.009949    0.76     0.5  0.12  0.0  
10  191.63    67.8  240.02  0.008403    0.76     0.5  0.52  0.0  
11  191.63    67.8   52.44  0.007368    0.76     0.5  0.08  0.0  
12  191.63    67.8   83.20  0.002651    0.76     0.5  0.23  0.0  
13  191.63    67.8  217.28  0.011575    0.76     0.5  0.84  0.0  
14  191.63    67.8  150.31  0.006893    0.76     0.5  0.32  0.0  
15  191.63    67.8  142.31  0.008539    0.76     0.5  0.87  0.0  
16  191.63    67.8  264.24  0.007938    0.76     0.5  0.04  0.0  
17  191.63    67.8  207.06  0.005622    0.76     0.5  0.01  0.0  
18  191.63    67.8  191.92  0.008050    0.76     0.5  0.57  0.0  
19  191.63    67.8  219.95  0.009465    0.76     0.5  0.06  0.0  
20  191.63    67.8  187.39  0.006713    0.76     0.5  0.97  0.0  
21  191.63  

# estimate convergence 

## 1. Rhat parameter

In [12]:
model_trace = glam_full.trace
rhats_params = az.rhat(model_trace, method="folded")

rhats_params_df = pd.DataFrame()
rhats_params_df['gamma'] = rhats_params.gamma.values
rhats_params_df['v'] = rhats_params.v.values
rhats_params_df['tau'] = rhats_params.tau.values
rhats_params_df['s'] = rhats_params.s.values

rhats_params_df  # if |rhat - 1 | < 0.05 (rhat: gelman-rubin statistic) the sampler converged 

/Users/pradyumna/anaconda3/lib/python3.7/site-packages/arviz/data/io_pymc3.py:89: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


gamma         v       tau         s
0   1.738165  1.186639  1.258079  1.053812
1   2.173723  1.073386  1.322334  1.025258
2   1.334411  1.695364  1.255432  1.560893
3   1.327429  1.123609  1.120194  1.100191
4   1.286346  1.295520  1.082180  1.091589
5   1.164065  1.206953  1.111541  1.118246
6   1.384741  1.422557  1.158603  1.070011
7   1.299173  1.137552  1.296579  1.144321
8   1.244390  1.130278  1.091183  1.031746
9   1.708875  1.490136  1.732245  1.055452
10  1.506514  1.494403  1.232121  1.103332
11  1.435639  1.390509  1.071399  1.491851
12  1.131030  1.333850  1.310828  1.971607
13  1.441978  1.198289  1.122589  1.026614
14  1.260737  1.335530  1.748998  1.138868
15  1.547820  1.101905  1.249992  1.095604
16  1.474744  1.072979  1.103621  1.034561
17  1.288350  1.202606  1.058500  1.109380
18  1.406265  1.120684  1.480552  1.057057
19  1.422416  1.103106  1.144513  1.028361
20  1.392582  1.190109  1.120458  1.061949
21  1.448734  1.223682  1.067834  1.024425
22  1.372783  1.465415  1.059800  1.166080
23  1.732325  1.150143  1.344181  1.035900
24  1.245253  1.139087  1.187028  1.108931
25  1.518559  1.092148  1.213002  1.056522
26  1.264293  1.623856  1.176622  1.683894
27  1.832332  1.384955  1.131628  1.160704
28  1.195309  1.151634  1.107632  1.175125
29  1.292214  1.054004  1.148345  1.091753
30  1.362608  1.199653  1.230533  1.158029
31  1.547720  1.057336  1.144088  1.026659

## 2. effective sample size

In [13]:
ess_model = az.ess(model_trace, relative=False)

ess_params_df = pd.DataFrame()
ess_params_df['gamma'] = ess_model.gamma.values
ess_params_df['v'] = ess_model.v.values
ess_params_df['tau'] = ess_model.tau.values
ess_params_df['s'] = ess_model.s.values

ess_params_df

gamma          v        tau           s
0    4.979555   8.318538   7.785854   15.629311
1    6.987034  17.156684  21.820910   26.114229
2    7.595718   9.242555   6.272830   10.747110
3   24.788681   7.992378  11.012336   26.315296
4    8.153438   8.908525  19.774377   28.270776
5   15.522435   7.471950  11.518790   11.500141
6   16.437503  17.248116  19.285776   34.679353
7   21.340958   7.527140   8.175543    9.079810
8   19.207144   8.944016   8.718288    9.337314
9   10.489565   6.430078   5.843272   30.575584
10   7.457206   5.388315  10.793149   12.086617
11  10.594239   7.499977  10.120281    7.011257
12   8.494762   6.101901   6.998893    4.703941
13   6.971056   7.208283  10.801007   25.711556
14  17.549522  10.460738   9.532189   22.576226
15   7.599106  16.223189   9.184682   16.951879
16  21.313904  18.796127  11.161740   63.318204
17  13.552022  13.577386  11.267331   59.032488
18   9.774890  12.679880   6.410284   33.498190
19   6.674954   9.877512  12.742051   16.364514
20   7.872079   7.710853   8.405185   11.197148
21  14.663115  30.341482  16.222897   70.959716
22  10.750890   6.243821  11.298229   11.908603
23   7.652832  10.826820  11.092686   14.993391
24  11.325661  14.491744  15.025963   16.835687
25  13.245151  16.892363   8.355400   22.147073
26  16.452764   5.656508   8.863667    6.660505
27   5.462094   6.956698  37.888331    9.693606
28   6.248341  10.469193  11.796359    8.706462
29  11.723354   8.500086  13.734700    9.966345
30  11.976736  12.207620   7.070986    8.889513
31   5.891280  33.772707  34.499800  103.524278

## 3. Percentage of divergence

In [14]:
# display the total number and percentage of divergent
divergent = model_trace['diverging']
print('Number of Divergent %d' % divergent.nonzero()[0].size)
divperc = divergent.nonzero()[0].size / len(model_trace) * 100
print('Percentage of Divergent %.1f' % divperc)

Number of Divergent 4
Percentage of Divergent 0.2


In [15]:
rhats_params_df.to_csv(str('results/convergence/GlamDataPF2019_hierarch_rhatsParams'+sufix+'.csv'))
ess_params_df.to_csv(str('results/convergence/GlamDataPF2019_hierarch_essParams'+sufix+'.csv'))

 # Waic scores (Less)

In [25]:
pm.waic(model_trace)

/Users/pradyumna/anaconda3/lib/python3.7/site-packages/arviz/data/io_pymc3.py:89: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/arviz/stats/stats.py:1415: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv

Computed from 8000 by 1 log-likelihood matrix

          Estimate       SE
elpd_waic -17833.35     0.00
p_waic      181.74        -

There has been a warning during the calculation. Please check the results.

The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if
you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive
accuracy.

In [22]:
model_waic = pm.waic(model_trace,scale = 'negative_log')
print ('Model WAIC',model_waic.waic)

Model WAIC 17833.34625021151


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/arviz/stats/stats.py:1415: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  "For one or more samples the posterior variance of the log predictive "


In [23]:
pm.loo(model_trace,scale = 'negative_log')

/Users/pradyumna/anaconda3/lib/python3.7/site-packages/arviz/stats/stats.py:683: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


Computed from 8000 by 1 log-likelihood matrix

          Estimate       SE
-elpd_loo 17711.52     0.00
p_loo        59.91        -

There has been a warning during the calculation. Please check the results.

The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if
you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive
accuracy.

In [24]:
np.save(str('results/waic/glam_PF2019_full'+ sufix +'.npy'), model_waic)

In [16]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_PF2019_full'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_full.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_full.waic = np.load(str('results/waic/glam_PF2019_full'+ sufix +'.npy'))

# Compute WAICs
np.save(str('results/waic/glam_PF2019_full'+ sufix +'.npy'), glam_full.waic)

Computing WAIC scores for full model...


TypeError: waic() got an unexpected keyword argument 'trace'

In [ ]:
glam_full.waic

In [ ]:
# Compute LOO

glam_full.loo = pm.loo(trace=glam_full.trace, model=glam_full.model)
glam_full.loo
np.save(str('results/loo/glam_PF2019_full'+ sufix +'.npy'), glam_full.loo)

In [ ]:
glam_full.loo

In [ ]:
# Predictions
print('Predicting test set data using full GLAM...')
glam_full.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_PF2019_full_hierarchical_cv'+sufix+'.csv')):
    glam_full.predict(n_repeats=50)
    glam_full.prediction.to_csv(str('results/predictions/glam_PF2019_full_hierarchical_cv'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical full GLAM predictions in "results/predictions". Skipping prediction...')
    glam_full.prediction = pd.read_csv(str('results/predictions/glam_PF2019_full_hierarchical_cv'+sufix+'.csv'))

glam_full.prediction.head()

### 1. no-bias GLAM

In [ ]:
# Fitting no-bias GLAM
print('Fitting no-bias GLAM hierarchically...')

glam_nobias = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_PF2019_nobias_hierarchical_cv'+sufix+'.npy')):
    glam_nobias.make_model('hierarchical', gamma_val=1.0, t0_val=0)
    glam_nobias.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_nobias.estimates = np.load(str('results/estimates/glam_PF2019_nobias_hierarchical_cv'+sufix+'.npy'))
 

In [ ]:
   
# Save parameter estimates
np.save(str('results/estimates/glam_PF2019_nobias_hierarchical_cv'+sufix+'.npy'), glam_nobias.estimates)
pd.DataFrame(glam_nobias.estimates)

In [ ]:
# In case it is already fitted
params_part_like = pd.DataFrame.from_dict(glam_nobias.estimates.item(0))
params_part_like

In [ ]:
# Compute LOO

glam_nobias.loo = pm.loo(trace=glam_nobias.trace, model=glam_nobias.model)
glam_nobias.loo

np.save(str('results/loo/glam_PF2019_nobias'+ sufix +'.npy'), glam_nobias.loo
)

In [ ]:
# Predictions
print('Predicting test set data using no-bias GLAM...')
glam_nobias.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_PF2019_nobias_hierarchical_cv'+sufix+'.csv')):
    glam_nobias.predict(n_repeats=50)
    glam_nobias.prediction.to_csv(str('results/predictions/glam_PF2019_nobias_hierarchical_cv'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical no-bias GLAM predictions in "results/predictions". Skipping prediction...')
    glam_nobias.prediction = pd.read_csv(str('results/predictions/glam_PF2019_nobias_hierarchical_cv'+sufix+'.csv'))

glam_nobias.prediction.head()

## 2. Plot fit

In [ ]:
print('Close Figure to continue...')
glam.plot_fit(test_data, [glam_full.prediction]);
#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);

plt.show()

## Parameters for full hierarchical model

In [ ]:
params_participant = glam_full.estimates
params_participant

In [ ]:
params_participant = pd.DataFrame.from_dict(glam_full.estimates.item(0))

In [ ]:
params_participant

In [ ]:
print ("Mean gamma " +  str(params_participant['gamma'].mean()))

In [ ]:
hist = params_participant[['SNR','gamma','tau','v']].hist(figsize = [20,3] , layout=[1,4],bins = 20)

## [END] 